In [1]:
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
from os import listdir
from os.path import isfile, join
import cv2

import scipy.io.wavfile
from spafe.utils import vis
from spafe.features.msrcc import msrcc

import librosa
import librosa.display
import numpy as np

import pylab

from nnAudio import Spectrogram
from scipy.io import wavfile
import torch
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join

from sklearn.model_selection import train_test_split

import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, Activation, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras.optimizers import SGD, RMSprop


import time

from nnAudio import Spectrogram
from scipy.io import wavfile
import torch
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join

Using TensorFlow backend.


In [2]:
print(os.getcwd())

/home/akshayrauthan0806/Research - PD


In [2]:
def create_Folder_Str(parentDir):
    
    # get the current working directory
    current_directory = os.getcwd()
    print(current_directory)

    # chunks directory
    parent_directory = current_directory + "/" + parentDir
    parent_read_directory = parent_directory + "/" + "Read Text"
    parent_read_hc_directory = parent_read_directory + "/" + "HC"
    parent_read_pd_directory = parent_read_directory + "/" + "PD"
    parent_sp_directory = parent_directory + "/" + "Spontaneous"
    parent_sp_hc_directory = parent_sp_directory + "/" + "HC"
    parent_sp_pd_directory = parent_sp_directory + "/" + "PD"


    # creating forlder for chunks.
    MYDIR = (parentDir)
    CHECK_FOLDER = os.path.isdir(MYDIR)

    # If folder doesn't exist, then create it.
    if not CHECK_FOLDER:
        #os.makedirs(MYDIR)
        os.makedirs(os.path.join(parentDir, 'Read Text', 'HC'))
        os.chdir(parent_read_directory)
        os.makedirs('PD')
        os.chdir(parent_directory)
        os.makedirs(os.path.join('Spontaneous', 'HC'))
        os.chdir(parent_sp_directory)
        os.makedirs('PD')
        os.chdir(current_directory)
        print("Folders Created")
    else:
        print(MYDIR, "folder already exists.")

In [4]:
create_Folder_Str('Spectograms')

/home/akshayrauthan0806/Research - PD
Folders Created


In [3]:
def getDir(type_of_text, parentDir):
    current_directory = os.getcwd()
    parent_directory = current_directory + "/" + parentDir
    parent_read_directory = parent_directory + "/" + "Read Text"
    parent_read_hc_directory = parent_read_directory + "/" + "HC"
    parent_read_pd_directory = parent_read_directory + "/" + "PD"
    parent_sp_directory = parent_directory + "/" + "Spontaneous"
    parent_sp_hc_directory = parent_sp_directory + "/" + "HC"
    parent_sp_pd_directory = parent_sp_directory + "/" + "PD"
    
    return {
        'ReadText_HC': parent_read_hc_directory,
        'ReadText_PD': parent_read_pd_directory,
        'Spontaneous_HC': parent_sp_hc_directory,
        'Spontaneous_PD': parent_sp_pd_directory,
    }[type_of_text]

In [4]:
def chunk_sound(filePath, type_of_text, parentDir, category):
    current_directory = os.getcwd()        
    files = [f for f in listdir(filePath) if isfile(join(filePath, f))]    
    destination_folder = getDir(type_of_text, parentDir)    

    cnt=0
    for file in files:
        os.chdir(current_directory)
        soundpath = filePath + "/" + file
        sound = AudioSegment.from_wav(soundpath)
        
        #chunking the audio based on time - 2 secs
        chunk_length_ms = 500 # pydub calculates in millisec
        chunks = make_chunks(sound, chunk_length_ms) #Make chunks of one sec
        
        #get the id of the person
        person_id = file.split('_')[0]
        
        #audio prefix
        audio_prefix = person_id + '_' + str(category)

        print(file, soundpath, len(chunks), person_id, category, audio_prefix)
        
        os.chdir(destination_folder)
        
        #checking if the Audio folder exists
        audioFolder = "Audio"
        audio_folder = os.path.isdir(audioFolder)
        
        melSpectrogramFolder = "MelSpectrogram"
        MelSpec_exists = os.path.isdir(melSpectrogramFolder)
        
        gammatonegramFolder = "Gammatonegram"
        Gammatone_exists = os.path.isdir(gammatonegramFolder)
        
        if audio_folder:
            print("Audio Folder already exists")
        else:
            os.makedirs(audioFolder)
            print("Folder created - ", audio_folder)
            
        if MelSpec_exists:
            print("MelSpectrogram Folder already exists")
        else:
            os.makedirs(melSpectrogramFolder)
            print("Folder created - ", melSpectrogramFolder)
            
        if Gammatone_exists:
            print("Gammatonegram Folder already exists")
        else:
            os.makedirs(gammatonegramFolder)
            print("Folder created - ", gammatonegramFolder)
            
        #export directory
        export_dir = destination_folder + '/' + audioFolder
        os.chdir(export_dir)
        
        for i, chunk in enumerate(chunks):
            chunk_name = (audio_prefix+"_"+"{0}.wav").format(cnt)
            chunk.export(chunk_name, format="wav")
            cnt=cnt+1
            
        os.chdir(current_directory)
        
    os.chdir(current_directory)

In [7]:
chunk_sound("./ProcessedAudio/Read Text/HC", "ReadText_HC", "Spectograms", 0)
chunk_sound("./ProcessedAudio/Read Text/PD", "ReadText_PD", "Spectograms", 1)
chunk_sound("./ProcessedAudio/Spontaneous/HC", "Spontaneous_HC", "Spectograms", 0)
chunk_sound("./ProcessedAudio/Spontaneous/PD", "Spontaneous_PD", "Spectograms", 1)

ID05_hc_0_0_0.wav ./ProcessedAudio/Read Text/HC/ID05_hc_0_0_0.wav 153 ID05 0 ID05_0
Folder created -  False
Folder created -  MelSpectrogram
Folder created -  Gammatonegram
ID35_hc_0_0_0.wav ./ProcessedAudio/Read Text/HC/ID35_hc_0_0_0.wav 110 ID35 0 ID35_0
Audio Folder already exists
MelSpectrogram Folder already exists
Gammatonegram Folder already exists
ID26_hc_0_0_0.wav ./ProcessedAudio/Read Text/HC/ID26_hc_0_0_0.wav 221 ID26 0 ID26_0
Audio Folder already exists
MelSpectrogram Folder already exists
Gammatonegram Folder already exists
ID11_hc_0_0_0.wav ./ProcessedAudio/Read Text/HC/ID11_hc_0_0_0.wav 184 ID11 0 ID11_0
Audio Folder already exists
MelSpectrogram Folder already exists
Gammatonegram Folder already exists
ID15_hc_0_0_0.wav ./ProcessedAudio/Read Text/HC/ID15_hc_0_0_0.wav 284 ID15 0 ID15_0
Audio Folder already exists
MelSpectrogram Folder already exists
Gammatonegram Folder already exists
ID14_hc_0_0_0.wav ./ProcessedAudio/Read Text/HC/ID14_hc_0_0_0.wav 222 ID14 0 ID14_0
Aud

ID00_hc_0_0_0.wav ./ProcessedAudio/Spontaneous/HC/ID00_hc_0_0_0.wav 124 ID00 0 ID00_0
Audio Folder already exists
MelSpectrogram Folder already exists
Gammatonegram Folder already exists
ID36_hc_0_0_0.wav ./ProcessedAudio/Spontaneous/HC/ID36_hc_0_0_0.wav 102 ID36 0 ID36_0
Audio Folder already exists
MelSpectrogram Folder already exists
Gammatonegram Folder already exists
ID25_hc_0_0_0.wav ./ProcessedAudio/Spontaneous/HC/ID25_hc_0_0_0.wav 139 ID25 0 ID25_0
Audio Folder already exists
MelSpectrogram Folder already exists
Gammatonegram Folder already exists
ID10_hc_0_0_0.wav ./ProcessedAudio/Spontaneous/HC/ID10_hc_0_0_0.wav 177 ID10 0 ID10_0
Audio Folder already exists
MelSpectrogram Folder already exists
Gammatonegram Folder already exists
ID09_hc_0_0_0.wav ./ProcessedAudio/Spontaneous/HC/ID09_hc_0_0_0.wav 181 ID09 0 ID09_0
Audio Folder already exists
MelSpectrogram Folder already exists
Gammatonegram Folder already exists
ID08_hc_0_0_0.wav ./ProcessedAudio/Spontaneous/HC/ID08_hc_0_0_0.w

In [2]:
# path joining version for other paths
DIR = '/home/akshayrauthan0806/Research - PD/Spectograms/Read Text/HC/Audio'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

# path joining version for other paths
DIR = '/home/akshayrauthan0806/Research - PD/Spectograms/Read Text/PD/Audio'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

# path joining version for other paths
DIR = '/home/akshayrauthan0806/Research - PD/Spectograms/Spontaneous/HC/Audio'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

# path joining version for other paths
DIR = '/home/akshayrauthan0806/Research - PD/Spectograms/Spontaneous/PD/Audio'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

4412
2649
2873
2136


In [9]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [10]:
get_available_gpus()

['/device:GPU:0']

In [11]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')
devices = tf.config.list_physical_devices('GPU')
print(len(devices))

Num GPUs Available:  1
1


In [12]:
# Selecting GPU
import torch
if torch.cuda.is_available():
    device = "cuda:0"
    
def findDevice():
    if torch.cuda.is_available():
        device = "cuda:0"
    return device

In [6]:
device = "cpu"
torch.set_default_tensor_type('torch.FloatTensor')

###### MelSpectrogram Generation

In [14]:
def melSpecLibrosa(filePath, savePath):
    files = [f for f in listdir(filePath) if isfile(join(filePath, f))]

    for file in files:
        complete_path = filePath + '/' + file        
        file_name = file.split('.')[0]
        save_path = savePath + '/' + file_name + '.jpeg'
        sig, fs = librosa.load(complete_path)
        S = librosa.feature.melspectrogram(y=sig, sr=fs)
        librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
        pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
        pylab.close()

###### Gammatone Spectrogram Generation

In [4]:
def Gammatone(filePath, savePath):
    files = [f for f in listdir(filePath) if isfile(join(filePath, f))]

    for file in files:
        complete_path = filePath + '/' + file
        file_name = file.split('.')[0]
        save_path = savePath + '/' + file_name + '.jpeg'
        sr, audio = wavfile.read(complete_path)
        duration_seconds = len(audio) / float(sr)
        if duration_seconds == 0.5:
            x = torch.tensor(audio, device=device).float()
            spec_layer = Spectrogram.Gammatonegram(sr=sr, n_fft=2048, n_bins=64, hop_length=512, 
                                                   window='hann', center=True, pad_mode='reflect', power=2.0, 
                                                   htk=True, fmin=0.0, fmax=None, norm=1, trainable_bins=False, 
                                                   trainable_STFT=False, verbose=False)
            spec_gamma = spec_layer(x)
            lib_s = librosa.power_to_db(spec_gamma, ref=np.max)
            lib_s = lib_s.reshape(lib_s.shape[1], lib_s.shape[2])
            librosa.display.specshow(lib_s)
            pylab.axis('off') # no axis
            pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
            pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
            pylab.close()
        else:
            print(file)

In [ ]:
os.getcwd()

In [16]:
melSpecLibrosa('/home/akshayrauthan0806/Research - PD/Spectograms/Read Text/HC/Audio', '/home/akshayrauthan0806/Research - PD/Spectograms/Read Text/HC/MelSpectrogram')
melSpecLibrosa('/home/akshayrauthan0806/Research - PD/Spectograms/Read Text/PD/Audio', '/home/akshayrauthan0806/Research - PD/Spectograms/Read Text/PD/MelSpectrogram')
melSpecLibrosa('/home/akshayrauthan0806/Research - PD/Spectograms/Spontaneous/HC/Audio', '/home/akshayrauthan0806/Research - PD/Spectograms/Spontaneous/HC/MelSpectrogram')
melSpecLibrosa('/home/akshayrauthan0806/Research - PD/Spectograms/Spontaneous/PD/Audio', '/home/akshayrauthan0806/Research - PD/Spectograms/Spontaneous/PD/MelSpectrogram')

ID26_0_279
ID15_0_799
ID10_0_2250
ID05_0_60
ID12_0_3418
ID03_0_3569
ID11_0_531
ID25_0_2061
ID03_0_3622
ID01_0_3777
ID12_0_3416
ID09_0_2413
ID12_0_3345
ID00_0_1670
ID11_0_656
ID21_0_1432
ID28_0_2974
ID11_0_521
ID25_0_2135
ID11_0_486
ID12_0_3339
ID23_0_4177
ID14_0_1169
ID21_0_1522
ID03_0_3570
ID19_0_1294
ID11_0_564
ID26_0_307
ID25_0_2004
ID05_0_31
ID19_0_1175
ID36_0_1823
ID00_0_1579
ID08_0_2772
ID23_0_4261
ID08_0_2651
ID26_0_331
ID00_0_1772
ID22_0_3957
ID28_0_2948
ID19_0_1283
ID08_0_2617
ID31_0_3022
ID23_0_4131
ID01_0_3748
ID21_0_1400
ID08_0_2584
ID15_0_849
ID03_0_3623
ID15_0_771
ID10_0_2387
ID31_0_3198
ID05_0_45
ID00_0_1554
ID09_0_2421
ID12_0_3328
ID28_0_2819
ID25_0_2178
ID26_0_385
ID25_0_2030
ID19_0_1292
ID23_0_4248
ID19_0_1293
ID01_0_3767
ID10_0_2352
ID36_0_1890
ID12_0_3384
ID15_0_933
ID36_0_1855
ID23_0_4181
ID14_0_1111
ID36_0_1893
ID35_0_243
ID36_0_1810
ID35_0_235
ID01_0_3722
ID08_0_2752
ID05_0_135
ID01_0_3761
ID15_0_769
ID19_0_1269
ID25_0_2190
ID26_0_355
ID15_0_750
ID11_0_484
ID36_0

ID23_0_4213
ID09_0_2440
ID36_0_1787
ID31_0_3132
ID03_0_3578
ID26_0_269
ID28_0_2908
ID36_0_1847
ID25_0_2003
ID10_0_2209
ID28_0_2907
ID28_0_2953
ID35_0_171
ID08_0_2587
ID28_0_2950
ID36_0_1949
ID03_0_3471
ID28_0_2925
ID22_0_3952
ID00_0_1675
ID23_0_4210
ID19_0_1374
ID00_0_1671
ID08_0_2694
ID22_0_4043
ID09_0_2475
ID03_0_3518
ID01_0_3810
ID01_0_3672
ID31_0_3002
ID22_0_4072
ID10_0_2357
ID00_0_1599
ID31_0_3116
ID11_0_550
ID26_0_405
ID25_0_2184
ID00_0_1626
ID31_0_3093
ID31_0_3104
ID12_0_3325
ID28_0_2964
ID26_0_321
ID08_0_2725
ID31_0_3167
ID09_0_2410
ID23_0_4281
ID11_0_613
ID31_0_3223
ID15_0_904
ID31_0_2994
ID10_0_2314
ID08_0_2612
ID23_0_4247
ID15_0_673
ID25_0_2019
ID11_0_526
ID10_0_2215
ID31_0_3062
ID22_0_3974
ID08_0_2674
ID12_0_3301
ID08_0_2687
ID15_0_699
ID05_0_53
ID25_0_1990
ID35_0_230
ID23_0_4322
ID10_0_2381
ID10_0_2372
ID19_0_1241
ID21_0_1430
ID21_0_1507
ID12_0_3433
ID15_0_734
ID03_0_3597
ID15_0_930
ID12_0_3317
ID23_0_4241
ID19_0_1317
ID03_0_3488
ID31_0_3234
ID10_0_2351
ID23_0_4290
ID10_0_

/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=529
  n_fft, y.shape[-1]


ID23_0_4359
ID23_0_4285
ID00_0_1738
ID31_0_3047
ID31_0_3131
ID22_0_3961
ID03_0_3643
ID01_0_3757
ID31_0_3225
ID11_0_512
ID10_0_2233
ID21_0_1520
ID01_0_3749
ID25_0_1976
ID25_0_2070
ID12_0_3408
ID01_0_3829
ID10_0_2227
ID31_0_3019
ID00_0_1703
ID01_0_3750
ID21_0_1488
ID22_0_3941
ID26_0_297
ID19_0_1286
ID26_0_349
ID11_0_637
ID26_0_345
ID01_0_3714
ID36_0_1800
ID21_0_1485
ID23_0_4334
ID08_0_2698
ID10_0_2244
ID21_0_1454
ID00_0_1729
ID31_0_3136
ID11_0_652
ID12_0_3404
ID14_0_1148
ID05_0_56
ID36_0_1945
ID05_0_18
ID08_0_2655
ID23_0_4231
ID19_0_1291
ID28_0_2926
ID25_0_2187
ID22_0_3907
ID26_0_338
ID05_0_142
ID03_0_3474
ID14_0_1070
ID00_0_1730
ID26_0_451
ID26_0_422
ID10_0_2206
ID31_0_3059
ID26_0_366
ID28_0_2833
ID25_0_2186
ID00_0_1595
ID22_0_3988
ID03_0_3521
ID36_0_1912
ID14_0_1092
ID22_0_4052
ID19_0_1193
ID22_0_4062
ID08_0_2721
ID09_0_2470
ID10_0_2317
ID08_0_2685
ID08_0_2662
ID00_0_1597
ID08_0_2690
ID10_0_2309
ID31_0_3000
ID05_0_73
ID03_0_3460
ID05_0_126
ID21_0_1451
ID19_0_1315
ID22_0_3968
ID08_0_275

ID01_0_3858
ID11_0_666
ID28_0_2839
ID26_0_305
ID03_0_3560
ID14_0_1088
ID14_0_1155
ID15_0_717
ID36_0_1779
ID23_0_4354
ID19_0_1380
ID01_0_3817
ID36_0_1795
ID08_0_2602
ID01_0_3784
ID12_0_3388
ID31_0_3008
ID05_0_24
ID12_0_3402
ID00_0_1641
ID19_0_1256
ID01_0_3889
ID08_0_2589
ID36_0_1872
ID03_0_3573
ID11_0_567
ID15_0_870
ID28_0_2888
ID22_0_4038
ID22_0_4016
ID25_0_2029
ID21_0_1413
ID23_0_4324
ID08_0_2631
ID01_0_3882
ID01_0_3674
ID00_0_1756
ID25_0_2158
ID25_0_2155
ID05_0_47
ID19_0_1341
ID01_0_3758
ID26_0_386
ID22_0_4025
ID28_0_2862
ID08_0_2719
ID23_0_4273
ID10_0_2270
ID28_0_2956
ID23_0_4136
ID28_0_2881
ID01_0_3736
ID22_0_4061
ID05_0_39
ID28_0_2877
ID22_0_4051
ID26_0_452
ID14_0_1105
ID15_0_834
ID03_0_3508
ID23_0_4196
ID23_0_4128
ID23_0_4259
ID10_0_2246
ID03_0_3631
ID15_0_749
ID28_0_2936
ID12_0_3351
ID26_0_375
ID03_0_3589
ID36_0_1809
ID19_0_1296
ID05_0_81
ID19_0_1340
ID22_0_3966
ID21_0_1476
ID28_0_2911
ID25_0_2048
ID08_0_2623
ID00_0_1618
ID21_0_1498
ID35_0_169
ID12_0_3387
ID11_0_609
ID21_0_1389


...

ID33_1_444
ID29_1_1149
ID02_1_866
ID24_1_356
ID32_1_856
ID27_1_705
ID16_1_106
ID34_1_717
ID33_1_387
ID33_1_457
ID30_1_1546
ID20_1_486
ID27_1_700
ID16_1_228
ID04_1_1712
ID16_1_235
ID13_1_1833
ID30_1_1586
ID29_1_1106
ID16_1_114
ID33_1_394
ID02_1_1018
ID06_1_1506
ID30_1_1522
ID07_1_1241
ID02_1_911
ID34_1_777
ID02_1_1021
ID32_1_818
ID32_1_846
ID04_1_1679
ID07_1_1275
ID27_1_690
ID07_1_1277
ID07_1_1405
ID07_1_1297
ID06_1_1516
ID24_1_329
ID16_1_158
ID24_1_366
ID32_1_790
ID27_1_699
ID20_1_521
ID02_1_1081
ID20_1_516
ID30_1_1547
ID34_1_748
ID02_1_974
ID02_1_885
ID13_1_1973
ID04_1_1717
ID30_1_1593
ID29_1_1145
ID33_1_421
ID30_1_1555
ID33_1_420
ID02_1_1103
ID20_1_476
ID16_1_160
ID13_1_2079
ID07_1_1251
ID17_1_1
ID04_1_1702
ID17_1_51
ID27_1_670
ID20_1_520
ID07_1_1262
ID27_1_581
ID04_1_1774
ID16_1_245
ID34_1_747
ID13_1_2097
ID33_1_404
ID07_1_1320
ID29_1_1125
ID06_1_1458
ID30_1_1572
ID16_1_126
ID06_1_1502
ID32_1_833
ID07_1_1323
ID07_1_1226
ID24_1_263
ID30_1_1554
ID02_1_1059
ID32_1_783
ID27_1_683
ID30_1

/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=2029
  n_fft, y.shape[-1]


ID30_1_1607
ID30_1_1542
ID02_1_865
ID20_1_477
ID02_1_904
ID07_1_1392
ID13_1_1859
ID27_1_677
ID34_1_737
ID16_1_196
ID27_1_611
ID13_1_2042
ID16_1_192
ID13_1_1997
ID27_1_691
ID27_1_556
ID33_1_396
ID04_1_1648
ID24_1_274
ID06_1_1486
ID13_1_1982
ID24_1_378
ID29_1_1166
ID24_1_346
ID27_1_542
ID04_1_1760
ID13_1_1944
ID17_1_12
ID16_1_143
ID24_1_273
ID06_1_1455
ID04_1_1720
ID02_1_1096
ID34_1_725
ID24_1_314
ID24_1_291
ID07_1_1321
ID20_1_468
ID02_1_937
ID13_1_2011
ID02_1_863
ID07_1_1217
ID13_1_1798
ID27_1_532
ID04_1_1734
ID16_1_156
ID24_1_264
ID13_1_1939
ID17_1_38
ID02_1_950
ID32_1_827
ID32_1_800
ID32_1_817
ID27_1_604
ID30_1_1524
ID04_1_1656
ID06_1_1489
ID13_1_2085
ID13_1_1846
ID07_1_1264
ID06_1_1512
ID32_1_781
ID33_1_458
ID06_1_1500
ID13_1_1867
ID16_1_215
ID04_1_1753
ID34_1_726
ID13_1_2108
ID13_1_2101
ID13_1_1842
ID27_1_661
ID07_1_1378
ID16_1_255


/opt/conda/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1521
  n_fft, y.shape[-1]


ID29_1_1152
ID02_1_1034
ID16_1_191
ID13_1_2030
ID30_1_1523
ID13_1_1924
ID16_1_176
ID16_1_194
ID29_1_1190
ID33_1_400
ID34_1_724
ID27_1_561
ID29_1_1171
ID06_1_1515
ID24_1_288
ID16_1_116
ID29_1_1110
ID27_1_616
ID02_1_876
ID02_1_920
ID27_1_709
ID30_1_1616
ID20_1_509
ID07_1_1227
ID04_1_1696
ID16_1_242
ID24_1_343
ID24_1_355
ID16_1_174
ID29_1_1164
ID02_1_1064
ID16_1_206
ID13_1_1988
ID02_1_1066
ID07_1_1354
ID24_1_320
ID33_1_402
ID04_1_1788
ID16_1_123
ID13_1_1800
ID34_1_720
ID33_1_424
ID24_1_272
ID07_1_1308
ID06_1_1448
ID07_1_1212
ID32_1_849
ID30_1_1535
ID32_1_804
ID32_1_779
ID13_1_1827
ID13_1_1883
ID32_1_839
ID02_1_917
ID06_1_1508
ID33_1_453
ID04_1_1795
ID30_1_1618
ID02_1_1010
ID06_1_1432
ID13_1_1802
ID27_1_551
ID24_1_265
ID02_1_965
ID16_1_218
ID13_1_1977
ID04_1_1692
ID07_1_1398
ID13_1_1930
ID02_1_940
ID04_1_1662
ID32_1_794
ID02_1_983
ID33_1_442
ID13_1_1909
ID34_1_754
ID04_1_1733
ID27_1_676
ID04_1_1770
ID32_1_802
ID27_1_586
ID30_1_1549
ID13_1_1903
ID06_1_1464
ID27_1_640
ID06_1_1418
ID27_1_597


In [7]:
Gammatone('/home/akshayrauthan0806/Research - PD/Spectograms/Read Text/HC/Audio', '/home/akshayrauthan0806/Research - PD/Spectograms/Read Text/HC/Gammatonegram')
Gammatone('/home/akshayrauthan0806/Research - PD/Spectograms/Read Text/PD/Audio', '/home/akshayrauthan0806/Research - PD/Spectograms/Read Text/PD/Gammatonegram')
Gammatone('/home/akshayrauthan0806/Research - PD/Spectograms/Spontaneous/HC/Audio', '/home/akshayrauthan0806/Research - PD/Spectograms/Spontaneous/HC/Gammatonegram')
Gammatone('/home/akshayrauthan0806/Research - PD/Spectograms/Spontaneous/PD/Audio', '/home/akshayrauthan0806/Research - PD/Spectograms/Spontaneous/PD/Gammatonegram')

ID32_1_857.wav
ID27_1_714.wav
ID02_1_1105.wav
ID24_1_380.wav
ID13_1_2135.wav
ID17_1_89.wav
ID29_1_1193.wav
ID20_1_525.wav
ID34_1_777.wav
ID06_1_1516.wav
ID30_1_1639.wav
ID33_1_467.wav
ID16_1_255.wav
ID04_1_1795.wav
ID07_1_1414.wav
